In [2]:
!pip install pandas scikit-learn


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 42.8 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: numpy
    Found existing installation: numpy 1.22.4
    Uninstalling numpy-1.22.4:
      Successfully uninstalled numpy-1.22.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
mkl-fft 1.3.10 requires mkl, which is not installed.
hdijupyterutils 0.21.0 requires pandas<2.0.0,>=0.17.1, but you have pandas 2.2.2 which is incompatible.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.1.1 which is incompatible.
sagemaker 2.229.0 requires numpy<2.0,>=1.9.0, but you have numpy 2.1.1 which is incompatible.
sparkmagic 0.21.0 requires pandas<2.0.0,>=0.17.1, but you have pandas 2.2.2 which is incompatible.


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Load the dataset
bucket = 'mlops-sagemaker-bucket-pb'
data_key = 'train.csv'
data_location = f's3://{bucket}/{data_key}'

df = pd.read_csv(data_location)

# Drop columns that won’t be useful for the model
df = df.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1)

# Fill missing values
df['Age'].fillna(df['Age'].mean(), inplace=True)
df['Embarked'].fillna(df['Embarked'].mode()[0], inplace=True)

# Encode categorical variables
le = LabelEncoder()
df['Sex'] = le.fit_transform(df['Sex'])
df['Embarked'] = le.fit_transform(df['Embarked'])

# Split dataset into features (X) and labels (y)
X = df.drop(['Survived'], axis=1)
y = df['Survived']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Save preprocessed data to S3 for training
X_train.to_csv('train_features.csv', index=False)
y_train.to_csv('train_labels.csv', index=False)
X_test.to_csv('test_features.csv', index=False)
y_test.to_csv('test_labels.csv', index=False)

# Upload preprocessed data to S3
import boto3
s3 = boto3.resource('s3')
s3.Bucket(bucket).upload_file('train_features.csv', 'train_features.csv')
s3.Bucket(bucket).upload_file('train_labels.csv', 'train_labels.csv')
s3.Bucket(bucket).upload_file('test_features.csv', 'test_features.csv')
s3.Bucket(bucket).upload_file('test_labels.csv', 'test_labels.csv')


In [7]:
import boto3

s3 = boto3.client('s3')
bucket_name = 'mlops-sagemaker-bucket-pb'

# Upload training data
s3.upload_file('train.csv', bucket_name, 'train/train.csv')

# Upload testing data
s3.upload_file('test.csv', bucket_name, 'test/test.csv')


FileNotFoundError: [Errno 2] No such file or directory: 'train.csv'